In [253]:
from EduKTM import DKT
import torch
import torch.utils.data as Data
import numpy as np
import logging
import tqdm
from sklearn.metrics import roc_auc_score

In [254]:
NUM_QUESTIONS = 512
BATCH_SIZE = 128
HIDDEN_SIZE = 10
NUM_LAYERS = 1

def get_data_loader(data_path, batch_size, shuffle=False):
    data = torch.FloatTensor(np.load(data_path))
    data_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    return data_loader

In [255]:
train_loader = get_data_loader(
    './train_data.npy', BATCH_SIZE, True)
test_loader = get_data_loader(
    './test_data.npy', BATCH_SIZE, False)

logging.getLogger().setLevel(logging.INFO)

dkt = DKT(NUM_QUESTIONS, HIDDEN_SIZE, NUM_LAYERS)

In [ ]:
dkt = DKT(NUM_QUESTIONS, HIDDEN_SIZE, NUM_LAYERS)
dkt.train(train_loader, epoch=20)
dkt.save("dkt.params")


In [ ]:
dkt.load("dkt.params")
auc = dkt.eval(test_loader)
print("auc: %.6f" % auc)

In [236]:
def process_raw_pred(raw_question_matrix, raw_pred, num_questions: int) -> tuple:
    questions = torch.nonzero(raw_question_matrix)[1:, 1] % num_questions
    length = questions.shape[0]
    pred = raw_pred[: length]
    pred = pred.gather(1, questions.view(-1, 1)).flatten()
    truth = torch.nonzero(raw_question_matrix)[1:, 1] // num_questions
    
    return pred, truth

def eval(model, test_data) -> float:
    model.eval()
    y_pred = torch.Tensor([])
    y_truth = torch.Tensor([])
    for batch in tqdm.tqdm(test_data, "evaluating"):
        integrated_pred = model(batch)
        batch_size = batch.shape[0]
        for student in range(batch_size):
            pred, truth = process_raw_pred(batch[student], integrated_pred[student], NUM_QUESTIONS)
            y_pred = torch.cat([y_pred, pred])
            y_truth = torch.cat([y_truth, truth])
    return [y_pred, y_truth]

In [ ]:
from torch import tensor


model = dkt.dkt_model

y_pred, y_truth = eval(model, test_loader)

print(y_pred[:10])
print(y_truth[:10])

y_pred_binary = (y_pred >= 0.5).float()
print("Binary predictions:", y_pred_binary[:10])

print("Accuracy:", torch.sum(y_pred_binary == y_truth)/len(y_truth))
print("y_pred_binary shape", y_pred_binary.shape)
roc_auc_score(y_truth.detach().numpy(), y_pred.detach().numpy())



In [ ]:

first_batch = next(iter(test_loader))
first_batch0 = first_batch[0]
oneStudent = first_batch0.unsqueeze(0)
print(oneStudent.shape)


model.eval()
pred = model(oneStudent)
pred, truth = process_raw_pred(oneStudent[0], pred[0], NUM_QUESTIONS)

print(truth)

# print(first_batch.shape)
# print(oneStudent.shape)

# print(oneStudent[0])
